<a href="https://colab.research.google.com/github/Anu-kriti/AA/blob/master/Gala_iteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [99]:
import pandas as pd
import numpy as np 
import tensorflow as tf
from numpy import load
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

import distutils
if distutils.version.LooseVersion(tf.__version__) < '1.14':
    raise Exception('This notebook is compatible with TensorFlow 1.14 or higher, for TensorFlow 1.13 or lower please use the previous version at https://github.com/tensorflow/tpu/blob/r1.13/tools/colab/fashion_mnist.ipynb')

df = load("Gala_clean_data.npz")
X, y = df['arr_0'], df['arr_1']

x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 1)

print(x_train.shape, y_train.shape)

(4188, 128, 128, 3) (4188, 4)


In [0]:
#p = np.load("Gala_test_set.csv.npy")
y_train = np.array([np.argmax(i) for i in y_train])
y_test = np.array([np.argmax(i) for i in y_test])


#y_train = tf.cast(y_train, tf.int32)
#y_test  = tf.cast(y_test , tf.int32)


In [0]:
def create_model():
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.BatchNormalization(input_shape=x_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.BatchNormalization(input_shape=x_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.BatchNormalization(input_shape=x_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(256))
  model.add(tf.keras.layers.Activation('elu'))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(4))
  model.add(tf.keras.layers.Activation('softmax'))
  return model


In [12]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu = 'grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)

tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)
strategy

INFO:tensorflow:Initializing the TPU system: 10.78.170.218:8470
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Querying Tensorflow master (grpc://10.78.170.218:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 17804153609752737110)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 10187493547592329860)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 370698102906664144)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 2683988688521008041)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/dev

In [0]:
"""x_train = tf.cast(x_train, tf.float32)
x_test  = tf.cast(x_test , tf.float32)"""

In [0]:
import os

with strategy.scope():
  model = create_model()
  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, ),
      loss='sparse_categorical_crossentropy',
      metrics=['sparse_categorical_accuracy'])

#model.summary()


In [102]:
y_train[:10]

array([0, 3, 2, 2, 2, 3, 2, 1, 2, 2])

In [101]:
model.fit(
    x_train, y_train,
    epochs=1
    #batch_size = 12,
    #steps_per_epoch=12,
    #validation_data=(x_test, y_test)
    #validation_freq=17
)
model.save_weights('./gala_first_itr.h5', overwrite=True)

InvalidArgumentError: ignored

In [0]:
LABEL_NAMES = ['Food', 'Attire', 'Decorationandsignage', 'misc']
